In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import sys
import random
from pprint import pprint

In [90]:
df = pd.read_csv('file2.csv')

In [91]:
# samples = df.drop(columns = {'new_cases'})

In [92]:
# labels = df['new_cases']

In [93]:
# from sklearn.model_selection import train_test_split

In [94]:
# train_samples, valid_samples, train_labels, valid_labels = train_test_split(samples, labels, test_size=0.2)

In [95]:
# train_samples, test_samples, train_labels, test_labels = train_test_split(train_samples, train_labels, test_size=0.2)

In [96]:
# train = pd.concat((train_samples,train_labels), axis = 1)
# test = pd.concat((test_samples, test_labels), axis = 1)
# valid = pd.concat((valid_samples, valid_labels), axis = 1)

In [97]:
# train.to_csv('train.csv',index=False)
# test.to_csv('test.csv',index=False)
# valid.to_csv('valid.csv',index=False)

In [98]:
train = pd.read_csv('train.csv')

In [99]:
train_samples = train.drop(columns = {'new_cases'})
train_labels = train['new_cases']

In [100]:
train_samples.head()

,retail_and_recreation,residential,transit_stations,parks,workplaces,total_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,reproduction_rate,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,stringency_index
0,-85.857,28.286,-66.143,-63.714,-59.857,24530.0,1454.000,780.0,59.0,42.000,...,1.41,41247.0,541789.0,0.393,0.030,29524.0,0.021,0.049,20.3,96.30
1,-39.143,13.429,-30.714,-45.286,-22.714,5562663.0,90346.714,88935.0,1053.0,1165.571,...,0.99,731534.0,64392594.0,46.661,0.530,1021881.0,0.740,0.088,11.3,83.80
2,-67.857,17.000,-43.143,-55.286,-32.857,216824.0,8391.143,6088.0,259.0,222.000,...,1.24,137158.0,4103233.0,2.973,0.099,123010.0,0.089,0.068,14.7,87.50
3,-27.429,11.429,-8.571,-14.857,-12.571,10450284.0,18045.571,150999.0,429.0,223.429,...,0.92,916951.0,180253315.0,130.618,0.664,901665.0,0.653,0.020,50.0,68.98
4,-76.143,21.571,-52.429,-57.857,-43.571,106475.0,4597.571,3302.0,146.0,126.714,...,1.32,101475.0,2404267.0,1.742,0.074,92098.0,0.067,0.050,20.0,81.94


In [322]:
# pip install tensorflow

In [430]:
from tensorflow.keras import models, layers
import tensorflow as tf

In [431]:
# Feature Extractor
def getFeatureRepresentation(x):
    tensors = tf.convert_to_tensor(x, dtype = float)
    model = models.Sequential()
    model.add(tf.keras.Input(shape=(26,)))
    model.add(layers.Dense(160, activation='tanh'))
    model.add(layers.Dense(80, activation='tanh'))
    model.compile()
    feature_representation = model.predict(x = tensors)
    return feature_representation

In [432]:
train_feature_repr = getFeatureRepresentation(train_samples)
val_feature_repr = getFeatureRepresentation(valid_samples)
test_feature_repr = getFeatureRepresentation(test_samples)

3/3 [==============================] - 0s 9ms/step


In [433]:
train_feature_repr.shape

(267, 80)

In [434]:
#Task Label Generator
def taskLabelGenerator(feature_repr, labels):
    
    labels = labels.to_numpy()
    labels = np.reshape(labels, (labels.size, 1))
    
    feaRepLabel = np.concatenate((feature_repr, labels), axis = 1)
    tensors = tf.convert_to_tensor(feaRepLabel, dtype = float)
    
    model = models.Sequential()
    model.add(tf.keras.Input(shape=(81,)))
    model.add(layers.Dense(160))
    model.add(layers.Dense(128))
    model.add(layers.Dense(80))
    model.compile()
    task_labels = model.predict(x = tensors)
    task_label_mean = np.average(task_labels, axis = 0)
    layer = layers.Softmax()
    task_label = layer(task_label_mean).numpy()
    return task_label

In [435]:
train_task_label = taskLabelGenerator(train_feature_repr, train_labels)

9/9 [==============================] - 0s 3ms/step


In [436]:
train_task_label.shape

(80,)

In [437]:
#Weights Generator
def weightsGenerator(feature_repr, labels, task_label):
    
    labels = labels.to_numpy()
    labels = np.reshape(labels, (labels.size, 1))
    
    task_label = np.reshape(task_label, (1,80))
    task_label = np.tile(task_label, (labels.size, 1))
    
    feaRepLabelTaskLabel = np.concatenate((feature_repr, labels, task_label), axis = 1)
    tensor = tf.convert_to_tensor(feaRepLabelTaskLabel, dtype = float)
    
    model = models.Sequential()
    model.add(tf.keras.Input(shape=(161,)))
    model.add(layers.Dense(256))  
    model.add(layers.Dense(164))
    model.add(layers.Dense(80))
    model.compile()
    weights = model.predict(x = tensor)
    weight_mean = np.average(weights, axis = 0)
    return weight_mean

In [438]:
weights = weightsGenerator(train_feature_repr, train_labels, train_task_label)

9/9 [==============================] - 0s 2ms/step


In [439]:
result = np.dot(train_feature_repr, weights)

In [440]:
from sklearn.metrics import mean_squared_error

In [441]:
rms = mean_squared_error(train_labels, result, squared=False)
rms

115629.65041137255

Adding Self Attention Block

In [183]:
pip install keras_self_attention

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for keras_self_attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18895 sha256=390feab15ba0047f253ddfb50017dff4bd1b928ce9cae34da07b80240f4e7e49
  Stored in directory: c:\users\administrator\appdata\local\pip\cache\wheels\78\c1\84\b83a2fd6f1d63e136cba74bac4126bee3b8705eef6486635fd
Successfully built keras_self_attention
Note: you may need to restart the kernel to use updated packages.


In [26]:
import keras
from keras_self_attention import SeqSelfAttention

In [30]:
def selfAttentionBlock(model, embedding, residual_connection):
    model.add(SeqSelfAttention(attention_activation = 'sigmoid'))
    model.add(layers.Dense(128, activation = 'relu'))
    model.add(layers.Dense(64))
    output = model.add(layers.LayerNormalization(axis=1))
    model.layers.Add()([embedding, output])
    model.compile()
    output = model.predict(embedding)
    return output

In [31]:
#Task Label Generator
def taskLabelGenerator(feature_repr, labels):
    
    labels = labels.to_numpy()
    labels = np.reshape(labels, (labels.size, 1))
    
    feaRepLabel = np.concatenate((feature_repr, labels), axis = 1)
    tensor = tf.convert_to_tensor(feaRepLabel, dtype = float)
    
    model = models.Sequential()
    
    #First Layer
    model.add(tf.keras.Input(shape=(41,)))
    
    #Second Layer
    embedding = model.add(layers.Dense(64))
    
    #First Self-Attention Block
    tensor = selfAttentionBlock(model, embedding, embedding)
    
    #Second Self-Attention Block
    tensor = selfAttentionBlock(model, tensor, embedding)
    
    #Third Self-Attention Block
    tensor = selfAttentionBlock(model, tensor, embedding)
    
    #Final Layer
    model.add(layer.Dense(40))
    
    model.compile()
    task_labels = model.predict(x = tensor)
    
    #Taking mean of all task labels
    task_label_mean = np.average(task_labels, axis = 0)
    
    #Applying Softmax
    layer = layers.Softmax()
    task_label = layer(task_label_mean).numpy()
    return task_label

In [32]:
train_task_label = taskLabelGenerator(train_feature_repr, train_labels)

IndexError: tuple index out of range

In [ ]:
#Weights Generator
def weightsGenerator(feature_repr, labels, task_label):
    
    labels = labels.to_numpy()
    labels = np.reshape(labels, (labels.size, 1))
    
    task_label = np.reshape(task_label, (1,40))
    task_label = np.tile(task_label, (labels.size, 1))
    
    feaRepLabelTaskLabel = np.concatenate((feature_repr, labels, task_label), axis = 1)
    tensor = tf.convert_to_tensor(feaRepLabelTaskLabel, dtype = float)
    
    model = models.Sequential()
    
    #First Layer
    model.add(tf.keras.Input(shape=(81,)))
    
    #Second Layer
    embedding = model.add(layers.Dense(64))
    
    #First Self-Attention Block
    tensor = selfAttentionBlock(embedding, embedding)
    
    #Second Self-Attention Block
    tensor = selfAttentionBlock(tensor, embedding)
    
    #Third Self-Attention Block
    tensor = selfAttentionBlock(tensor, embedding)
    
    #Final Layer
    model.add(layers.Dense(40))
    
    model.compile()
    weights = model.predict(x = tensor)
    
    #Taking mean of all the weights
    weight_mean = np.average(weights, axis = 0)
    return weight_mean